In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


In [3]:
trades = pd.read_csv('../data/raw/historical_data.csv')
sentiment = pd.read_csv('../data/raw/fear_greed_index.csv')


In [4]:
print("Trader Data Shape:", trades.shape)
print("Sentiment Data Shape:", sentiment.shape)

print("\nTrader Data Info:")
display(trades.info())

print("\nSentiment Data Info:")
display(sentiment.info())


Trader Data Shape: (211224, 16)
Sentiment Data Shape: (2644, 4)

Trader Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Account           211224 non-null  object 
 1   Coin              211224 non-null  object 
 2   Execution Price   211224 non-null  float64
 3   Size Tokens       211224 non-null  float64
 4   Size USD          211224 non-null  float64
 5   Side              211224 non-null  object 
 6   Timestamp IST     211224 non-null  object 
 7   Start Position    211224 non-null  float64
 8   Direction         211224 non-null  object 
 9   Closed PnL        211224 non-null  float64
 10  Transaction Hash  211224 non-null  object 
 11  Order ID          211224 non-null  int64  
 12  Crossed           211224 non-null  bool   
 13  Fee               211224 non-null  float64
 14  Trade ID          211224 non-null

None


Sentiment Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       2644 non-null   int64 
 1   value           2644 non-null   int64 
 2   classification  2644 non-null   object
 3   date            2644 non-null   object
dtypes: int64(2), object(2)
memory usage: 82.8+ KB


None

In [5]:
# Missing values
trades_missing = trades.isna().sum()
sentiment_missing = sentiment.isna().sum()

# Duplicates
trades_duplicates = trades.duplicated().sum()
sentiment_duplicates = sentiment.duplicated().sum()

print("Trader missing values:\n", trades_missing)
print("\nSentiment missing values:\n", sentiment_missing)

print("\nTrader duplicates:", trades_duplicates)
print("Sentiment duplicates:", sentiment_duplicates)


Trader missing values:
 Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64

Sentiment missing values:
 timestamp         0
value             0
classification    0
date              0
dtype: int64

Trader duplicates: 0
Sentiment duplicates: 0
